In [19]:
import torch.nn as nn
import torch

In [20]:
random_tensor = torch.randint(1, 256, (1,3, 640, 640), dtype=torch.float32)

In [21]:
random_tensor.shape

torch.Size([1, 3, 640, 640])

In [22]:
class Conv2d(nn.Module):
    def __init__(self,in_channels, out_channels, kernel_size, padding, stride = 1):
        super(Conv2d, self).__init__()
        self.conv = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, padding=padding, stride=stride)
    def forward(self, x):
        x = self.conv(x)
        x = nn.BatchNorm2d(x.shape[1], affine=False)(x)
        x = nn.SiLU()(x)
        return x
        

In [23]:
class Bottleneck(nn.Module):
    def __init__(self,shortcut,in_channels):
        super(Bottleneck, self).__init__()
        self.conv1_bottleneck = Conv2d(in_channels=in_channels, out_channels=in_channels // 2, kernel_size=3, padding=1, stride=1).to(dtype=torch.float32)
        self.conv2_bottleneck = Conv2d(in_channels=in_channels // 2, out_channels=in_channels, kernel_size=3, padding=1, stride=1).to(dtype=torch.float32)
        self.shortcut = shortcut


    def forward(self, x):
        y = x.clone()
        x = self.conv1_bottleneck(x)
        x = self.conv2_bottleneck(x)
        
        if self.shortcut: 
            x = x + y
            
        return x

In [24]:
class C2f(nn.Module):
    def __init__(self,n,shortcut,in_channels):
        super(C2f, self).__init__()
        self.n = n
        self.shortcut = shortcut
        self.conv1 = Conv2d(in_channels=in_channels, out_channels=in_channels, kernel_size=1, padding=0, stride=1).to(dtype=torch.float32)
        self.bottleneck_blocks = nn.ModuleList([Bottleneck(self.shortcut, in_channels // 2) for _ in range(self.n)])
        self.conv2 = Conv2d(in_channels=(((self.n+2)*in_channels)//2), out_channels= in_channels , kernel_size=1, padding=0, stride=1).to(dtype=torch.float32)

    def forward(self, x):
        x = self.conv1(x)
        x1, x2 = torch.chunk(x, 2, dim=1)
        x2 = torch.cat((x2, x1), dim=1)

        for bottleneck_block in self.bottleneck_blocks:
             x1 = bottleneck_block(x1)
             x2 = torch.cat((x2, x1), dim=1)
        
        x = x2
        x = self.conv2(x)
        return x

In [25]:
class SPPF(nn.Module):
    def __init__(self, in_channels):
        super(SPPF, self).__init__()
        self.conv1 =  Conv2d(in_channels=in_channels, out_channels=in_channels, kernel_size=1, padding=0, stride=1)
        self.conv2 =  Conv2d(in_channels=in_channels, out_channels=in_channels, kernel_size=1, padding=0, stride=1)
        self.pool_sizes = [5, 9, 13]


    def forward(self, x):
        x = self.conv1(x)
        pooled_outputs = []
#         x1 = torch.cat((x, y), dim=1)
        for pool_size in self.pool_sizes:
            pooled = nn.functional.max_pool2d(x, kernel_size=pool_size)
            pooled_outputs.append(pooled)
        else:
            pooled_outputs.append(x)
        
        
        x = torch.cat(pooled_outputs, dim=1)
        x = self.conv2(x)
        print("1=", x.shape)
        return x

In [26]:
class Backbone(nn.Module):
    def __init__(self):
        super(Backbone, self).__init__()

        self.p1 = Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1, stride=2).to(dtype=torch.float32)
        self.p2 = Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1, stride=2).to(dtype=torch.float32)
        
        self.c2f_1 = C2f(1, True,  32)
        
        self.p3 =  Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1, stride=2).to(dtype=torch.float32)
        self.c2f_2 = C2f(2, True,  64)
        
        self.p4 =  Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1, stride=2).to(dtype=torch.float32)
        self.c2f_3 = C2f(2, True,  128)
        self.p5 =  Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1, stride=2).to(dtype=torch.float32)
        self.c2f_4 = C2f(2, True,  256)

        self.sppf = SPPF(256)
        
    def forward(self, x):
        x = self.p1(x)
        x = self.p2(x)
        x = self.c2f_1(x) 
        x = self.p3(x)
        y = self.c2f_2(x)
        x = self.p4(y)
        z = self.c2f_3(x)
        x = self.p5(z)
        x = self.c2f_4(x)
        x = self.sppf(x)
        
        return y, z, x

In [27]:
random_tensor = torch.randint(1, 256, (1,3, 640, 640), dtype=torch.float32)


In [28]:
b = Backbone()
a,c,e = b(random_tensor)

RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 4 but got size 2 for tensor number 1 in the list.

In [ ]:
a.shape

In [ ]:
c.shape

In [ ]:
e.shape